<a href="https://colab.research.google.com/github/Asato65/fm/blob/master/%E3%83%9F%E3%82%AB%E3%83%B32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Googleドライブに接続
# ミカンフォルダをドライブ直下にコピーしておく
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from PIL import Image

BASE_PATH = '/content/drive/MyDrive/ミカン/all/learning'
IMG_FOLDER = f'{BASE_PATH}/img'
DATA_FILE = f'{BASE_PATH}/data.txt'

# モデルによってイメージサイズが変わる
IMG_SIZE = 224          # @param {type: "number"}

# テスト画像の分割比率（0.1ならテスト画像に1割、残り9割をトレーニングに使用）
TEST_SIZE = 0.3         # @param {type: "number"}
# テスト画像とトレーニング画像に分割する際のシード値（固定）
RANDOM_STATE = 0

# エポック数（学習回数） （エポック数ととりあえず馬鹿みたいに上げてlossとかいろいろみて値を設定してあげて過学習を防ぐ）
EPOCHS = 200             # @param {type: "number"}
# 検証データの分割割合（過学習を防止するデータ）
VALIDATION_SPLIT = 0.2  # @param {type: "number"}

# 予測する値の範囲
PREDICTION_MIN = 8      # @param {type: "number"}
PREDICTION_MAX = 19     # @param {type: "number"}

In [4]:
# 糖度データと画像データを読み込む
# ミカン番号の順番通りに配列にデータが入る

with open(DATA_FILE, 'r') as f:
    lines = f.readlines()
    str_data = lines[0].replace('\ufeff', '')
    float_data = [float(x) for x in str_data.split(',')]
    datas = np.array(float_data)

# 画像データの読み込み
images = []
for img_name in sorted(os.listdir(IMG_FOLDER), key=lambda x: int(x.split('.')[0])):
    img_path = os.path.join(IMG_FOLDER, img_name)
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.image.per_image_standardization(img)  # 画像の標準化
    img = tf.cast(img, tf.float32) / 255.0  # 画像の正規化
    images.append(img)

images = np.array(images)

In [5]:
# データを学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(images, datas, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# シンプルなモデル
# モデルの構築
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
  tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
  tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(1)
])
# 今回はyが5個値を持ってるのでDense(5)にして値を5つに絞る

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=["mae"])

# モデルの学習
model.fit(X_train, y_train, epochs=EPOCHS, validation_split=VALIDATION_SPLIT)  # 検証データを使用

@tf.function
def predict_images(images):
    return model(images)

# 予測実行
predictions = predict_images(X_test)
# predictions = model.predict(X_test)
predictions = np.clip(predictions, PREDICTION_MIN, PREDICTION_MAX)


Epoch 1/200
2/2 [==============================] - 24s 10s/step - loss: 138.1551 - mae: 11.6312 - val_loss: 43.6874 - val_mae: 6.5295
Epoch 2/200
2/2 [==============================] - 0s 238ms/step - loss: 49.3891 - mae: 6.8312 - val_loss: 1.1294 - val_mae: 0.8011
Epoch 3/200
2/2 [==============================] - 0s 236ms/step - loss: 10.0889 - mae: 2.5401 - val_loss: 17.1915 - val_mae: 4.0172
Epoch 4/200
2/2 [==============================] - 0s 238ms/step - loss: 17.5986 - mae: 3.8070 - val_loss: 1.0700 - val_mae: 0.8129
Epoch 5/200
2/2 [==============================] - 0s 240ms/step - loss: 6.2103 - mae: 1.9082 - val_loss: 14.2119 - val_mae: 3.6278
Epoch 6/200
2/2 [==============================] - 0s 253ms/step - loss: 10.2195 - mae: 2.6802 - val_loss: 1.2371 - val_mae: 0.9112
Epoch 7/200
2/2 [==============================] - 0s 236ms/step - loss: 5.0655 - mae: 1.7508 - val_loss: 4.5112 - val_mae: 1.9266
Epoch 8/200
2/2 [==============================] - 0s 269ms/step - loss: 6

In [8]:
max_error = 0
# 結果の表示
for actual, predicted in zip(y_test, predictions):
    print(f'実際の糖度: {actual}, 予測された糖度: {predicted[0]}')
    max_error = abs(actual - predicted[0]) if max_error < abs(actual - predicted[0]) else max_error

_, mae = model.evaluate(X_test, y_test)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print(f'平均絶対誤差: {mae}')        # この誤差は％単位でないので注意
print(f'平均絶対パーセント誤差: {mape}')
print(f'最大誤差: {max_error}')     # この誤差は％単位でないので注意

実際の糖度: 11.2, 予測された糖度: 11.128643035888672
実際の糖度: 16.4, 予測された糖度: 13.996099472045898
実際の糖度: 17.8, 予測された糖度: 11.546449661254883
実際の糖度: 10.7, 予測された糖度: 9.491839408874512
実際の糖度: 12.0, 予測された糖度: 10.51966667175293
実際の糖度: 14.2, 予測された糖度: 13.815982818603516
実際の糖度: 11.9, 予測された糖度: 10.342164993286133
実際の糖度: 16.8, 予測された糖度: 12.42675495147705
実際の糖度: 13.5, 予測された糖度: 9.908897399902344
実際の糖度: 10.3, 予測された糖度: 9.777984619140625
実際の糖度: 12.4, 予測された糖度: 11.975240707397461
実際の糖度: 15.0, 予測された糖度: 12.158785820007324
実際の糖度: 13.5, 予測された糖度: 12.399106979370117
実際の糖度: 13.2, 予測された糖度: 12.974185943603516
実際の糖度: 13.2, 予測された糖度: 11.101628303527832
実際の糖度: 12.0, 予測された糖度: 11.424145698547363
実際の糖度: 12.6, 予測された糖度: 12.192231178283691
実際の糖度: 12.2, 予測された糖度: 11.062195777893066
実際の糖度: 16.6, 予測された糖度: 13.181197166442871
実際の糖度: 12.0, 予測された糖度: 10.356489181518555
実際の糖度: 10.8, 予測された糖度: 10.611156463623047
実際の糖度: 10.2, 予測された糖度: 11.03058910369873
実際の糖度: 11.3, 予測された糖度: 11.912714004516602
実際の糖度: 10.6, 予測された糖度: 11.36996841430664
実際の糖度: 11.8, 予測された糖度: 9

現在の問題：予測糖度がおかしな値になる（20以上や、10以下の値、負の値をとる）、原因は不明

まずは、予測した糖度が10-15に収まるように修正したい

→範囲指定をした

また、誤差をなくすために

* 過学習を防ぐ
* 画像をグレースケールにしてみる、色情報を減らす
* 予測方法を変えてみる
* クリスマスのときの測定データも使ってデータを増やす
* 画像サイズを変える
* ミカンのサイズを使ってみる（例えば：画像と大きさ（大中小）から糖度を予測してみる、画像から大きさを予測、その大きさから糖度を予測してみる、など）<br>（画像＆ほかの測定データを組み合わせるか、画像から糖度以外の測定データを予測し、そのデータから糖度を予測する）

とかは思いついた


グレースケール、データを増やす、サイズを変える、などはやってる
